In [2]:
import os
import requests
import pandas as pd
from bs4 import BeautifulSoup
from tqdm.notebook import tqdm

In [3]:
BASE_LINK = 'https://www.rmj.ru'

In [4]:
r = requests.get(f'{BASE_LINK}/articles')

In [5]:
base_soup = BeautifulSoup(r.content, parser='lxml')

In [6]:
categories = [l['href'] for l in base_soup.find('div', {'class': 'left-side'}).find_all('a', href=True)]
categories = [f'{BASE_LINK}/articles{l.replace("/articles", "")}' for l in categories if l.startswith('/art')][1:]

In [7]:
def add_article_to_copus(art_l, corpus):
    try:
        art_r = requests.get(art_l)
        art_s = BeautifulSoup(art_r.content, parser='lxml')
        header = art_s.find('div', {'class': 'h2'}).text
        annotation = art_s.find('div', {'id': 'resume-lang-ru'}).text
        text = art_s.find('div', {'itemprop': 'articleBody'}).text
        corpus.append([header, annotation, text])
    except Exception as e:
        print('\n', art_l)

In [8]:
corpus = []
for cat_l in tqdm(categories[:1]):
    print(cat_l, end='\r')
    cat_r = requests.get(cat_l)
    cat_soup = BeautifulSoup(cat_r.content, parser='lxml')
    last_page = int(cat_soup.find_all('li', {'class': 'page'})[-1].find('a', href=True)['href'].split('/')[-1][9:])
    pages = [cat_l] + [f'{cat_l}?PAGEN_2={i}' for i in range(2, last_page+1)]
    # print(pages)
    # break
    for page_l in tqdm(pages):
        page_r = requests.get(page_l)
        page_s = BeautifulSoup(page_r.content, parser='lxml')
        articles_links = [f"{BASE_LINK}/articles{l.find('a', href=True)['href']}" for l in page_s.find_all('div', {'class': 'article-sm'})]
        for art_l in articles_links:
            add_article_to_copus(art_l, corpus)

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/16 [00:00<?, ?it/s]

In [9]:
corpus_df = pd.DataFrame(corpus, columns=['header', 'annotation', 'text'])

In [10]:
corpus_df

,header,annotation,text
0,Влияние пандемии COVID-19 на психологическое б...,\nЦель исследования: оценка уровня психологиче...,\nВведение\r\n Психологическое благополучие ли...
1,SARS-CоV-2: что позволило вирусу вызвать длите...,\nЗавершившаяся пандемия COVID-19 привела к за...,\nВведение\r\n Коронавирусы представляют собой...
2,Факторы риска развития COVID-19: иммунологичес...,"\n\r\n\t Исследования последних лет, посвященн...",\n\r\nВведение \n\r\n\t Новая коронавирусная и...
3,К вопросу об антивирусной терапии COVID-19,\nВ обзоре представлены данные о заболеваемост...,\n Острые респираторные вирусные инфекц...
4,Листериозный менингоэнцефалит: особенности теч...,\nПроблема листериоза многие годы привлекает в...,\nВведение\r\n Проблема листериоза продолжает ...
...,...,...,...
118,"Боль, падения, ограничение двигательной активн...","\n\nВ статье обсуждаются актуальные, с точки з...",\n Введение \n\r\n\t Согласно данным ВОЗ ожида...
119,Международные рекомендации по ведению пациенто...,\n\nУсловия пандемии коронавирусной инфекции C...,\n\n\n\n\r\n\t Коронавирусная инфекция COVID-1...
120,Профилактика и лечение острых респираторных ин...,\n\nОстрые респираторные инфекции (ОРИ) являют...,\n Введение \n\r\n\t Острые респираторные инфе...
121,COVID-19 и дети,\n\n11 марта 2020 г. ВОЗ объявила о начале пан...,\n\n\n\n\r\n\t 11 марта 2020 г. ВОЗ объявила о...


In [14]:
corpus_df['annotation']

0      \nЦель исследования: оценка уровня психологиче...
1      \nЗавершившаяся пандемия COVID-19 привела к за...
2      \n\r\n\t Исследования последних лет, посвященн...
3      \nВ обзоре представлены данные о заболеваемост...
4      \nПроблема листериоза многие годы привлекает в...
                             ...                        
118    \n\nВ статье обсуждаются актуальные, с точки з...
119    \n\nУсловия пандемии коронавирусной инфекции C...
120    \n\nОстрые респираторные инфекции (ОРИ) являют...
121    \n\n11 марта 2020 г. ВОЗ объявила о начале пан...
122    \n\nВнебольничная пневмония (ВП) является одно...
Name: annotation, Length: 123, dtype: object

In [15]:
corpus_df['text']

0      \nВведение\r\n Психологическое благополучие ли...
1      \nВведение\r\n Коронавирусы представляют собой...
2      \n\r\nВведение \n\r\n\t Новая коронавирусная и...
3      \n        Острые респираторные вирусные инфекц...
4      \nВведение\r\n Проблема листериоза продолжает ...
                             ...                        
118    \n Введение \n\r\n\t Согласно данным ВОЗ ожида...
119    \n\n\n\n\r\n\t Коронавирусная инфекция COVID-1...
120    \n Введение \n\r\n\t Острые респираторные инфе...
121    \n\n\n\n\r\n\t 11 марта 2020 г. ВОЗ объявила о...
122    \n\n\n\n Введение \n\r\n\t Внебольничная пневм...
Name: text, Length: 123, dtype: object

In [11]:
import nltk

In [17]:
# nltk.word_tokenize(corpus_df['annotation'][0])

In [18]:
#corpus_df.to_csv('rmj_corpus_raw.csv')

In [12]:
def standardize_text(df, an_field):
    df[an_field] = df[an_field].str.replace(r"http\S+", "")
    df[an_field] = df[an_field].str.replace(r"http", "")
    df[an_field] = df[an_field].str.replace(r"@\S+", "")
    df[an_field] = df[an_field].str.replace(r"[^,!?@\'\`\"\_\t\r\n]", " ")
    df[an_field] = df[an_field].str.replace(r"@", "at")
    df[an_field] = df[an_field].str.replace(r"[\n\t\r]", "")
    df[an_field] = df[an_field].str.lower()
    return df

corpus = standardize_text(corpus_df, "annotation")

corpus.head()

,header,annotation,text
0,Влияние пандемии COVID-19 на психологическое б...,\nцель исследования: оценка уровня психологиче...,\nВведение\r\n Психологическое благополучие ли...
1,SARS-CоV-2: что позволило вирусу вызвать длите...,\nзавершившаяся пандемия covid-19 привела к за...,\nВведение\r\n Коронавирусы представляют собой...
2,Факторы риска развития COVID-19: иммунологичес...,"\n\r\n\t исследования последних лет, посвященн...",\n\r\nВведение \n\r\n\t Новая коронавирусная и...
3,К вопросу об антивирусной терапии COVID-19,\nв обзоре представлены данные о заболеваемост...,\n Острые респираторные вирусные инфекц...
4,Листериозный менингоэнцефалит: особенности теч...,\nпроблема листериоза многие годы привлекает в...,\nВведение\r\n Проблема листериоза продолжает ...


In [13]:
def standardize_text(df, text_field):
    df[text_field] = df[text_field].str.replace(r"http\S+", "")
    df[text_field] = df[text_field].str.replace(r"http", "")
    df[text_field] = df[text_field].str.replace(r"@\S+", "")
    df[text_field] = df[text_field].str.replace(r"[^,!?@\'\`\"\_\t\r\n]", " ")
    df[text_field] = df[text_field].str.replace(r"@", "at")
    df[text_field] = df[text_field].str.replace(r"[\n\t\r]", "")
    df[text_field] = df[text_field].str.lower()
    return df

corpus = standardize_text(corpus_df, "text")

corpus.head()

,header,annotation,text
0,Влияние пандемии COVID-19 на психологическое б...,\nцель исследования: оценка уровня психологиче...,\nвведение\r\n психологическое благополучие ли...
1,SARS-CоV-2: что позволило вирусу вызвать длите...,\nзавершившаяся пандемия covid-19 привела к за...,\nвведение\r\n коронавирусы представляют собой...
2,Факторы риска развития COVID-19: иммунологичес...,"\n\r\n\t исследования последних лет, посвященн...",\n\r\nвведение \n\r\n\t новая коронавирусная и...
3,К вопросу об антивирусной терапии COVID-19,\nв обзоре представлены данные о заболеваемост...,\n острые респираторные вирусные инфекц...
4,Листериозный менингоэнцефалит: особенности теч...,\nпроблема листериоза многие годы привлекает в...,\nвведение\r\n проблема листериоза продолжает ...


In [14]:
def standardize_text(df, head_field):
    df[head_field] = df[head_field].str.replace(r"http\S+", "")
    df[head_field] = df[head_field].str.replace(r"http", "")
    df[head_field] = df[head_field].str.replace(r"@\S+", "")
    df[head_field] = df[head_field].str.replace(r"[^,!?@\'\`\"\_]", " ")
    df[head_field] = df[head_field].str.replace(r"@", "at")
    df[head_field] = df[head_field].str.replace(r"[\n\t\r]", "")
    df[head_field] = df[head_field].str.lower()
    return df

corpus = standardize_text(corpus_df, "header")

corpus.to_csv("clean_corpusdata.csv")
corpus.head()

,header,annotation,text
0,влияние пандемии covid-19 на психологическое б...,\nцель исследования: оценка уровня психологиче...,\nвведение\r\n психологическое благополучие ли...
1,sars-cоv-2: что позволило вирусу вызвать длите...,\nзавершившаяся пандемия covid-19 привела к за...,\nвведение\r\n коронавирусы представляют собой...
2,факторы риска развития covid-19: иммунологичес...,"\n\r\n\t исследования последних лет, посвященн...",\n\r\nвведение \n\r\n\t новая коронавирусная и...
3,к вопросу об антивирусной терапии covid-19,\nв обзоре представлены данные о заболеваемост...,\n острые респираторные вирусные инфекц...
4,листериозный менингоэнцефалит: особенности теч...,\nпроблема листериоза многие годы привлекает в...,\nвведение\r\n проблема листериоза продолжает ...


In [22]:
corpus.tail()

,header,annotation,text
118,"боль, падения, ограничение двигательной активн...","\n\nв статье обсуждаются актуальные, с точки з...",\n введение \n\r\n\t согласно данным воз ожида...
119,международные рекомендации по ведению пациенто...,\n\nусловия пандемии коронавирусной инфекции c...,\n\n\n\n\r\n\t коронавирусная инфекция covid-1...
120,профилактика и лечение острых респираторных ин...,\n\nострые респираторные инфекции (ори) являют...,\n введение \n\r\n\t острые респираторные инфе...
121,covid-19 и дети,\n\n11 марта 2020 г. воз объявила о начале пан...,\n\n\n\n\r\n\t 11 марта 2020 г. воз объявила о...
122,практические рекомендации по ведению пациентов...,\n\nвнебольничная пневмония (вп) является одно...,\n\n\n\n введение \n\r\n\t внебольничная пневм...


In [15]:
from nltk.tokenize import RegexpTokenizer

tokenizer = RegexpTokenizer(r'\w+')

corpus["tokens_head"] = corpus["header"].apply(tokenizer.tokenize)
corpus["tokens_an"] = corpus["annotation"].apply(tokenizer.tokenize)
corpus["tokens_text"] = corpus["text"].apply(tokenizer.tokenize)
corpus.head()

,header,annotation,text,tokens_head,tokens_an,tokens_text
0,влияние пандемии covid-19 на психологическое б...,\nцель исследования: оценка уровня психологиче...,\nвведение\r\n психологическое благополучие ли...,"[влияние, пандемии, covid, 19, на, психологиче...","[цель, исследования, оценка, уровня, психологи...","[введение, психологическое, благополучие, личн..."
1,sars-cоv-2: что позволило вирусу вызвать длите...,\nзавершившаяся пандемия covid-19 привела к за...,\nвведение\r\n коронавирусы представляют собой...,"[sars, cоv, 2, что, позволило, вирусу, вызвать...","[завершившаяся, пандемия, covid, 19, привела, ...","[введение, коронавирусы, представляют, собой, ..."
2,факторы риска развития covid-19: иммунологичес...,"\n\r\n\t исследования последних лет, посвященн...",\n\r\nвведение \n\r\n\t новая коронавирусная и...,"[факторы, риска, развития, covid, 19, иммуноло...","[исследования, последних, лет, посвященные, ос...","[введение, новая, коронавирусная, инфекция, co..."
3,к вопросу об антивирусной терапии covid-19,\nв обзоре представлены данные о заболеваемост...,\n острые респираторные вирусные инфекц...,"[к, вопросу, об, антивирусной, терапии, covid,...","[в, обзоре, представлены, данные, о, заболевае...","[острые, респираторные, вирусные, инфекции, ор..."
4,листериозный менингоэнцефалит: особенности теч...,\nпроблема листериоза многие годы привлекает в...,\nвведение\r\n проблема листериоза продолжает ...,"[листериозный, менингоэнцефалит, особенности, ...","[проблема, листериоза, многие, годы, привлекае...","[введение, проблема, листериоза, продолжает, п..."


In [16]:
corpus.to_csv("rmj_corpus_tokenized.csv")

In [24]:
import keras

In [25]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical

In [26]:
all_words = [word for tokens in corpus["tokens_an"] for word in tokens]
VOCAB = sorted(list(set(all_words)))
print("%s words total, with a vocabulary size of %s" % (len(all_words), len(VOCAB)))

27057 words total, with a vocabulary size of 6991


In [27]:
all_words_1 = [word for tokens in corpus["tokens_text"] for word in tokens]
VOCAB_1 = sorted(list(set(all_words_1)))
print("%s words total, with a vocabulary size of %s" % (len(all_words_1), len(VOCAB_1)))

316299 words total, with a vocabulary size of 34354


In [32]:
summ = int(27057) + int(316299)

In [33]:
print(summ)

343356
